# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

### Send HTTP request to the target Website

In [ ]:
base_url="https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/"
URL=[]
for i in range(1,15):
  URL.append(base_url+str(i)+"/")

check the response status code

In [ ]:
for url in URL:
  source=requests.get(url)
  print(source.status_code)
  soap=BeautifulSoup(source.text,"html.parser")
  #print(soap.prettify())

200
200
200
200
200
200
200
200
200
200
200
200
200
200


### Parse the response content with BeautifulSoap

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

In [ ]:
def extract_product_characteristics_as_dict(product):
  title=product.select_one(".wd-entities-title a").text
  price=product.select(".woocommerce-Price-amount")
  regular_price=price[0].text
  if len(price)>1:
    discount_price=price[1].text
  else:
    discount_price=None
  page_link=product.select_one(".wd-entities-title a")["href"]
  add_to_cart_link=product.select_one(".wd-add-btn a")["href"]
  product_dict={
    "ProductTitle":title,
    "RegularPrice":regular_price,
    "DiscountPrice":discount_price,
    "ProductPage":page_link,
    "AddToCart":add_to_cart_link
  }
  return(product_dict)

In [ ]:
parsed_products=[]
for url in URL:
  source=requests.get(url)
  soap=BeautifulSoup(source.text,"html.parser")
  products=soap.select(".product-grid-item")
  for product in products:
    result=extract_product_characteristics_as_dict(product)
    parsed_products.append(result)

Repeat the extraction process for each page of products

In [ ]:
source=requests.get(URL[0])
soap=BeautifulSoup(source.text,"html.parser")
products=soap.select(".product-grid-item")
product=products[2]
title=product.select_one(".wd-entities-title a").text
price=product.select(".woocommerce-Price-amount")
regular_price=price[0].text
link=product.select_one(".wd-entities-title a")["href"]
product.select_one(".wd-add-btn a")["href"]

'?add-to-cart=20826'

### Create a pandas DataFrame with the scraped products

In [ ]:
df=pd.DataFrame(parsed_products)
df.head(10)

,ProductTitle,RegularPrice,DiscountPrice,ProductPage,AddToCart
0,Acer A315-23-A7KD,17.590 ден,None,https://clevershop.mk/product/acer-a315-23-a7kd/,?add-to-cart=21494
1,Acer A315-23-R5P2,27.490 ден,None,https://clevershop.mk/product/acer-a315-23-r5p2/,?add-to-cart=21510
2,ACER Aspire 1 A115-22,18.999 ден,15.999 ден,https://clevershop.mk/product/acer-aspire-1-nx...,?add-to-cart=20826
3,Acer Aspire 3 A315-23-R26A,29.990 ден,None,https://clevershop.mk/product/acer-aspire-3-a3...,?add-to-cart=21516
4,Acer Aspire 3 A315-58-33WK,24.490 ден,None,https://clevershop.mk/product/21498/,?add-to-cart=21498
5,Acer Aspire 3 A315-58-33WK,25.990 ден,None,https://clevershop.mk/product/acer-aspire-3-a3...,?add-to-cart=21506
6,ACER Aspire 5 (A515-56-35KA),23.990 ден,None,https://clevershop.mk/product/acer-aspire-5-a5...,?add-to-cart=21693
7,ACER ASPIRE 5 A515-45,29.990 ден,None,https://clevershop.mk/product/acer-aspire-5-a5...,?add-to-cart=21523
8,Acer Aspire 5 A515-45-R07Y Silver,24.990 ден,None,https://clevershop.mk/product/acer-aspire-5-a5...,?add-to-cart=21501
9,Acer Aspire 5 A515-45-R1FG,36.990 ден,None,https://clevershop.mk/product/acer-aspire-5-a5...,?add-to-cart=18324


Save the dataframe as `.csv`

In [ ]:
file_path = '/content/drive/MyDrive/Lab1 Scraping.csv'
df.to_csv(file_path,index=False)
